In [ ]:
# Inport dependencies
!pip install langchain
!pip install google_genai
!pip install langchain_experimental
!pip install langchain_google_genai
!pip install pypdf

ERROR: Could not find a version that satisfies the requirement google_genai (from versions: none)
ERROR: No matching distribution found for google_genai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
import sys
# from gemini import key

Model and API Key

In [ ]:
#load environment variables
load_dotenv()

#set the model name for our LLMs
GEMINI_MODEL = "gemini-pro"

#Store the API key in a variable
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
#Initialize the model
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model_name=GEMINI_MODEL
    temperature=0.8,
)

Kevin's Code:


In [ ]:
import requests
import pandas as pd
from googletrans import Translator
import openai


In [ ]:
# Additional imports for API chain.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import APIChain


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
Gemini_API_KEY = os.getenv("Gemini_API_KEY")


In [ ]:
# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"


In [ ]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=Gemini_API_KEY, model=GEMINI_MODEL, temperature=0.9)


In [ ]:
# Function to generate relocation insights using Gemini
def generate_relocation_insights(city):
    prompt = f"""
    Provide detailed insights about relocating to {city}. Include the following:
    1. Cultural aspects and lifestyle.
    2. Job opportunities and key industries.
    3. Cost of living overview.
    4. Any challenges or considerations for relocation.
    """
    response = llm.predict(prompt)
    return response


In [ ]:
from langchain.chains import APIChain

# Define functions for external API calls
def fetch_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"]
        }
    else:
        return {"error": f"Failed to fetch weather data: {response.status_code}"}



In [ ]:
# Chain Gemini model and OpenWeather API
def combined_relocation_info(city):
    # Fetch weather data
    weather_info = fetch_weather_data(city)
    if "error" in weather_info:
        weather_summary = weather_info["error"]
    else:
        weather_summary = (
            f"The current temperature in {city} is {weather_info['temperature']}°C "
            f"with {weather_info['description']}."
        )


In [ ]:
    # Generate relocation insights
    relocation_prompt = f"""
    Consider the following context: {weather_summary}.
    Provide detailed relocation insights for {city} including:
    - Cultural aspects.
    - Job opportunities.
    - Cost of living.
    """
    relocation_insights = llm.predict(relocation_prompt)
    return {
        "weather_summary": weather_summary,
        "relocation_insights": relocation_insights
    }


In [ ]:
def main():
    city = input("Enter a city to get relocation insights: ")

    # Fetch insights
    try:
        results = combined_relocation_info(city)
        print("\nWeather Summary:")
        print(results["weather_summary"])
        print("\nRelocation Insights:")
        print(results["relocation_insights"])
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()
